In [1]:
import pandas as pd
import numpy as np
from datetime import date

import pandas_datareader.data as web
import pandas_datareader as pdr

In [2]:
START_DATE = '1999-01-01'
END_DATE = '2021-10-31'
TICKS_FX = ['DEXUSUK','DEXUSEU','DEXSZUS','DEXJPUS']
TICKS_RF = ['USD1MTD156N','GBP1MTD156N','EUR1MTD156N','CHF1MTD156N','JPY1MTD156N']

rfraw = pdr.DataReader(TICKS_RF,data_source='fred', start=START_DATE,end=END_DATE)
fxraw = pdr.DataReader(TICKS_FX,data_source='fred', start=START_DATE,end=END_DATE)

In [3]:
info = pd.DataFrame(columns=['Ticker Description'])
info.loc['USD1M'] = '1-Month LIBOR, USD'
info.loc['GBP1M'] = '1-Month LIBOR, GBP'
info.loc['EUR1M'] = '1-Month LIBOR, EUR'
info.loc['CHF1M'] = '1-Month LIBOR, CHF'
info.loc['JPY1M'] = '1-Month LIBOR, JPY'

info.loc['USUK'] = 'Spot FX, US per UK (GBP)'
info.loc['USEU'] = 'Spot FX, US per EU (EUR)'
info.loc['USSZ'] = 'Spot FX, US per SZ (CHF)'
info.loc['USJPY'] = 'Spot FX, US per JP (JPY)'
info

,Ticker Description
USD1M,"1-Month LIBOR, USD"
GBP1M,"1-Month LIBOR, GBP"
EUR1M,"1-Month LIBOR, EUR"
CHF1M,"1-Month LIBOR, CHF"
JPY1M,"1-Month LIBOR, JPY"
USUK,"Spot FX, US per UK (GBP)"
USEU,"Spot FX, US per EU (EUR)"
USSZ,"Spot FX, US per SZ (CHF)"
USJPY,"Spot FX, US per JP (JPY)"


In [4]:
FREQ = 'M'
    
if FREQ == 'M':
    SCALE = 30
elif FREQ == 'Y':
    SCALE = ANNUALIZATION
elif FREQ == 'D':
    SCALE = 1
else:
    error('Frequency needs specified.')

In [5]:
rf = pd.DataFrame(index=rfraw.index)
for tick in TICKS_RF:
    lab = tick[0:5]
    rf[lab] = rfraw[tick]/100
    
    if 'GBP' in lab:
        ANNUALIZATION = 365
    else:
        ANNUALIZATION = 360        
    
    rf[lab] *= (SCALE/ANNUALIZATION)

rf = rf.resample(FREQ).last()
rf

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.004116,0.004938,0.002598,0.000995,0.000335
1999-02-28,0.004135,0.004624,0.002600,0.001036,0.000232
1999-03-31,0.004114,0.004434,0.002492,0.000993,0.000143
1999-04-30,0.004085,0.004404,0.002141,0.000801,0.000099
1999-05-31,0.004120,0.004420,0.002141,0.000833,0.000075
...,...,...,...,...,...
2021-06-30,0.000084,0.000046,-0.000488,-0.000675,-0.000050
2021-07-31,0.000075,0.000043,-0.000481,-0.000663,-0.000053
2021-08-31,0.000069,0.000042,-0.000486,-0.000661,-0.000059


In [6]:
fx = pd.DataFrame(index=fxraw.index)

for tick in TICKS_FX:
    lab = tick[-4:]
    if lab[-2:] == 'US':
        lab = lab[-2:] + lab[-4:-2]
        fx[lab] = 1/fxraw[tick]
    else:
        fx[lab] = fxraw[tick]
    
fx = fx.resample('M').last()
fx

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,1.6457,1.1371,0.705816,0.008621
1999-02-28,1.6027,1.0995,0.689893,0.008425
1999-03-31,1.6140,1.0808,0.676819,0.008444
1999-04-30,1.6085,1.0564,0.655437,0.008373
1999-05-31,1.6020,1.0422,0.654450,0.008273
...,...,...,...,...
2021-06-30,1.3806,1.1848,1.080731,0.009005
2021-07-31,1.3913,1.1864,1.103631,0.009116
2021-08-31,1.3747,1.1800,1.091584,0.009087


# Save to Excel

In [7]:
with pd.ExcelWriter('fx_carry_data_v2.xlsx') as writer:  
    info.to_excel(writer, sheet_name = 'descriptions')
    rf.to_excel(writer, sheet_name= 'risk-free rates')
    fx.to_excel(writer, sheet_name='fx rates')